# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.
  
![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

In [ ]:
%pip install python-dotenv

In [ ]:
# working directory 아래 `.env` file을 생성
# .gitignore file에 `.env`를 등록해서 github에 올라가지 않도록 함.
# 이름-값 형식으로 환경변수를 설정
## HUGGINGFACE_API_KEY = "받은 Access Token"

In [ ]:
# 환경변수 값 읽기
import os
os.getenv("JAVA_HOME")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
# working directory에서 환경변수 파일(default : `.env`)를 찾아서 그 file에 설정된 값을 O/S의 환경변수로 등록해준다.

In [ ]:
# 얘는 재시작하면 날아가서 위 코드로 다시 등록해줘야함
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
# os.environ["HUGGINGFACE_API_KEY"]
print(hf_api_key)

# Naver 영화댓글 분류

# Huggingface Dataset 패키지
- 허깅페이스 허브에 공유된 데이터셋을  다운로드해서 전처리 및 관리할 수있도록 돕는 라이브러리. 
- 많은 공개데이터셋을 동일한 인터페이스로 사용할 수있다.
- 설치
    - `pip install datasets`
- https://huggingface.co/datasets
- https://github.com/huggingface/datasets
      
## Huggingface Dataset loading
- datasets 로딩
    - `load_data('dataset name')`
        - huggingface datasets에 등록된 Dataset 이름 넣어 Loading한다.
          
![img](figures/huggingface_dataset.png)

In [ ]:
# NSMC dataset load
import datasets
from datasets import load_dataset

nsmc = load_dataset("e9t/nsmc", trust_remote_code=True)

In [ ]:
# Dataset
# DatasetDict : Dataset들을 모아놓은 dict기반의 자료구조
#				train/valid/test set을 모아서 제공할 때 사용

In [ ]:
type(nsmc), nsmc

In [ ]:
nsmc.keys()

In [ ]:
trainset = nsmc['train']
testset = nsmc['test']

In [ ]:
trainset

In [ ]:
# feature 조회
trainset['id'][:5], trainset['document'][:5], trainset['label'][:5]

In [ ]:
# datasets.Dataset을 다른 형식으로 변환
# dataset객체.to_xxxxx()
df = trainset.to_pandas()
df.head()

In [ ]:
# 다른 형식으로 저장된 data를 datasets.Dataset으로 변환
# datasets.Dataset.from_xxxxx()
d = datasets.Dataset.from_pandas(df.head(100))	# df 100개만 가져오기도 가능
d

In [ ]:
# Sampling 
# train : 10_000, test : 5_000
sample_train = trainset.shuffle().select(range(10_000))	# dataset.shuffle() : 섞어줌
sample_test = testset.shuffle().select(range(5_000))

In [ ]:
print(sample_train)
print(sample_test)

## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_id = "beomi/kcbert-base"
# tokenizer, 분류 모델 로딩 - tokenizer와 model은 같은 id를 받아야함
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = 2)
# num_labels : 분류할 class 개수 (긍/부정)
# model_id : kcbert-base - pretrained된 Feature Extractor
# model : Estimator는 학습 안된 layer로 구성

In [ ]:
print(model)

In [ ]:
train_X = sample_train["document"]	# sample_train -> trainset(전체)
train_y = sample_train["label"]

test_X = sample_test["document"]	# sample_train -> testset(전체)
test_y = sample_test["label"]

In [ ]:
# X(댓글)을 토큰화
train_encoding = tokenizer(train_X, return_tensors= "pt", padding=True)	# train_X에서 가장 긴 문장을 기준으로 padding 처리
test_encoding = tokenizer(test_X, return_tensors= "pt", padding=True)

In [ ]:
train_encoding.keys()
train_encoding['input_ids'].shape

In [ ]:
#################################
# Pytorch의 Dataset을 정의, 생성
#################################
import torch
from torch.utils.data import Dataset
class NSMCDataset(Dataset):
    
	def __init__(self, comments, labels):
		"""
		Args:
			comments(dict) : tokenizer로 토큰화한 input data
			labels(list) : 정답
		"""
		self.comments = comments
		self.labels = labels

	def __len__(self):
		return len(self.labels)

	def __getitem__(self, index):
		"""
		index번 째 data를 반환
		BERT model 입력 형식에 맞춰서 반환. input_ids, token_type_ids, attention_mask + label
		Args:
			index(int)
		Returns:
			dictionary - input_idx, token_type_ids, attention_mask, label
						 입력 data와 정답 label을 dict에 묶어서 반환
		"""
		data = {key:value[index] for key, value in self.comments.items()}
		data["labels"] = torch.tensor([self.labels[index]], dtype = torch.int64)
		return data

In [ ]:
train_set = NSMCDataset(train_encoding, train_y)	# NSMCDataset 객체 생성 (train set)
test_set = NSMCDataset(test_encoding, test_y)		# NSMCDataset 객체 생성 (test set)

## pytorch Dataset 생성
모델 입력으로 다음 4개 항목을 dictionary로 묶어서 제공하도록 구현한다.
1. input_ids: 입력 text 토큰을 id로 변환한 값
2. token_type_ids: 문자쌍 구분시 사용. 단일 문장: 0, 문자쌍-첫문장: 0, 두 번째 문장: 1
3. attention_mask: 실제 토큰값과 패딩구분값
4. labels: 정답 class index

1 ~ 3은 위의 train_encoding, test_encoding으로 만듬. labels은 train_data/test_data의 label 키 값 사용

# 학습
- Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공한다.
- TrainingArguments Trainer를 위한 설정을 하는 클래스
- TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision등을 쉽게 설정해 학습, 평가를 모두 진행할 수 있다.

In [ ]:
from transformers import TrainingArguments, Trainer

N_EPOCHS = 1
BATCH_SIZE = 64

# 어떻게 학습할지에 대한 설정
train_args = TrainingArguments(
    output_dir="models/nsmc", 	# train model을 저장할 dir path
    num_train_epochs=N_EPOCHS,	# train epoch수
    per_device_train_batch_size=BATCH_SIZE,	# train batch size
    per_device_eval_batch_size=BATCH_SIZE,	# eval batch size
    
	eval_strategy="epoch",	# 평가 전략 : "no", "steps", "epoch"
    save_strategy="epoch",	# 저장 전략
    
	save_total_limit=1,		# 저장할 model의 최대 개수
    load_best_model_at_end=True,		# 학습 종료 후 검증결과가 가장 좋은 모델을 저장.
										# True : eval_strategy, save_strategy가 같아야함.
	metric_for_best_model= "eval_loss",	# best model을 결할 검증 기준 평가지표
    greater_is_better=False,			# 검증 평가지표값이 높아야 좋은지 낮아야 좋은지
    
	report_to ="none"
)

In [ ]:
# 평가 함수 정의 - evaluate 패키지를 이용
# huggingface에서 제공하는 라이브러리, 다양한 평가함수들을 제공
%pip install evaluate scikit-learn

In [ ]:
import evaluate

# 정확도 평가 함수
acc_fn = evaluate.load("accuracy")	# f1, recall, precision

In [ ]:
pred = torch.tensor([0,1,0,1])
ref = torch.tensor([0,1,0,0])

acc_fn.compute(predictions=pred, references=ref)

In [ ]:
def compute_metrics(pred):
    """
    model이 학습하는 도중 예측값을 받아서 평가 점수를 계산하는 func
    Trainer에 의해서 호출된 func
    Args:
		pred(EvalPrediction) - model의 예측값, 정답들을 묶어서 제공
	Returns:
		dictionary - key: 평가지표이름, value: 평가점수
    """
    labels = pred.label_ids		# 정답
    preds = pred.predictions.argmax(axis=-1)		# model의 출력값
    metrics1 = evaluate.load("accuracy")
    metrics2 = evaluate.load("f1")
    
    acc = metrics1.compute(references = labels, predictions = preds)
    f1 = metrics2.compute(references = labels, predictions = preds)
    return {"accuracy" : acc, "f1" : f1}

In [ ]:
# Trainer 객체
trainer = Trainer(
    model = model,				# 학습 모델 대상
    args = train_args,			# TrainingArguments
    train_dataset=train_set,	# 학습 dataset, pythorch의 Dataset객체
    eval_dataset=test_set,		# 검증 dataset
    compute_metrics = compute_metrics	# loss 이외에 검증/평가 시 확인할 지표를 계산하는 func
)

In [ ]:
# trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# 저장 - local directory
## tokenizer와 model을 같이 같은 경로에 저장
save_path = "models/nsmc"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

## 모델 로드

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

save_path = "models/nsmc"
load_tokenizer = AutoTokenizer.from_pretrained(save_path)
load_model = AutoModelForSequenceClassification.from_pretrained(save_path)

# 추론

In [ ]:
sentence = ["이걸 영화라고 만든 거냐?", "아무 기대 없이 봤는데 재미있네.", "내가 감독이어도 이것보다 재미있게 만들겠다.", "시간이 어떻게 가는 줄 모르고 봤다."]

In [ ]:
from transformers import pipeline
pipe = pipeline(task = "text-classification", tokenizer = load_tokenizer, model = load_model)
result = pipe(sentence)

In [ ]:
result

In [ ]:
###### 학습한 model, tokenizer를 huggingface hub에 업로드 ######
# 1. TrainingArguments에 설정해서 학습 도중 끝나면 자동으로 업로드 되게 설정
# 2. model/tokenizer.push_to_hub("계정/모델id")를 이용해서 수동으로 올린다.

# 로그인
from huggingface_hub import login
# login() # token에 Access Token 입력 후 Login 클릭
login("Huggingface Access Token")
load_dotenv()

In [ ]:
import os
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
login(hf_api_key)

In [ ]:
model_id = "kcbert-nsmc-10000"
load_tokenizer.push_to_hub(model_id)
load_model.push_to_hub(model_id)